In [1]:
#!pipenv install numpy --dev

In [2]:
#!pipenv install pandas --dev

In [3]:
import vaex
from ast import literal_eval

In [4]:
df = vaex.from_csv("millio.5_song.csv")

In [5]:
import numpy as np

# Get edgelist

In [6]:
def list_check(i):
    if type(i)==list:
        return i
def extract_links(list_of_tuples):
    """
    1. List of tuples was saved as a string, turn it back into a list
    2. Turn Tuples into list. Output is a list of strings & lists
    3. filter only the lists, drop the identifiers like "Written by". Output is a list of lists. 
    4. Turn the list of lists into one list
    """
    a = literal_eval(list_of_tuples)
    a = list(filter(list_check,[i[1] for i in a]))
    a = [j.replace("https://genius.com/","").replace("artists/","").replace("-lyrics","") for i in a for j in i ]
    return a

In [7]:
%%time
df.add_virtual_column("meta_data_1",df.side_table.apply(extract_links))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 217 µs


In [8]:
md1 = df.evaluate(df.meta_data_1)

In [9]:
md1

array([list(['Pete-rock', 'Nas', 'Jamey-staub', 'Tony-dawsey', 'Columbia-records', 'Ahmad-jamal-trio-i-love-music', 'T-la-rock-and-jazzy-jay-its-yours', 'Nyck-caution-jamal', 'Theory-hazit-what-lies-beneath', 'Jay-z-dead-presidents-ii', 'Kendrick-lamar-and-jay-rock-dead-presidents-iii', 'Gamemast15r-my-world-is-dead-downfall', 'Eminem-jealousy-woes-ii', 'Junior-mafia-oh-my-lord', 'Hemp-gru-droga', 'Ras-kass-goldyn-chyld', 'Fabolous-the-world-is-yours-freestyle', 'Madchild-junior', 'Skyzoo-finesse-everything', 'Nataanii-means-dead-presidents-resented-me', 'Thundamentals-el-mundo', 'Nas-i-am-intro', 'Lazareth-dead-presidents', 'Necro-smooth-crimy', 'Corleone-p-its-mine', 'Omar-el-hachemi-and-e-herve-one-love', 'Mac-miller-nikes-on-my-feet', 'Kaydy-cain-6-feet-deep', 'Freddie-gibbs-thug-psalms', 'T-la-rock-and-jazzy-jay-its-yours', 'Joey-bada-amerikkkan-idol', 'Nas-thiefs-theme', 'Mc-wojtu-swiat-jest-nasz-historia-marka-pt-1', 'Mattick-four-more-years', 'B-dolan-kitchen-sink', 'Logic-dead

In [10]:
h = df.evaluate(df.header_links.apply(literal_eval))

In [11]:
def return_longest(a,b):
    if len(a)>len(b):
        return a
    else:
        return b

In [12]:
df.add_column("meta_data_2",np.array([return_longest(a,b) for a,b in zip(md1,h)]))

In [13]:
associated_data = df.evaluate(df.meta_data_2)

In [14]:
df.add_virtual_column("song_identifier",df.url.str.replace("https://genius.com/","").str.replace("-lyrics","").str.lower())

In [15]:
def edge_list(song_name,associated_data):
    return np.array([(song_name,i) for i in associated_data])

In [16]:
df.add_virtual_column("nodes",df.meta_data_2.apply(len))

In [17]:
song = df.song_identifier.evaluate()
data = df.meta_data_2.evaluate()

In [18]:
df.add_column("edge_list",np.array([edge_list(a,b) for a,b in zip(song,data)]))

In [19]:
edge_list = df.edge_list.evaluate()

In [20]:
edge_list  = np.array([ i for sublist in edge_list for i in sublist ])

# Get lyric embeddings

In [21]:
import cython

In [22]:
df.add_virtual_column("fixed_lyrics", df.lyrics.str.replace(r'\[(.+?)\]|[\']',"",regex=True).str.lower())

In [23]:
df.fixed_lyrics

Expression = fixed_lyrics
Length: 1,571,000 dtype: str (virtual column)
---------------------------------------------
      0  ', , "(Its yours)", Whose world is this?, The wo...
      1  ', I was sitting back, and I was thinking man, "...
      2  ', "Every morning I wake up, Im thanking God for...
      3  ', "Yeah, uh-huh, watch this yall", "Uhh, watch ...
      4  ', "Yea, thats right", "Its not a Hardy Boy myst...
                            ...                             
1570995
1570996  ', "Im wearin my colors: red shirt, red Stars an...
1570997  ', I woke up one morning, Ran outside because I ...
1570998  ', I wake up in the morning pop my clip up in my...
1570999  ', I feel like busting a cap in the face of a Sl...

In [24]:
z = df.fixed_lyrics.evaluate()

In [25]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

INFO:MainThread:gensim.summarization.textcleaner:'pattern' package not found; tag filters are not available for English


In [26]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(z)]

In [65]:
model = Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [67]:
model.build_vocab(documents)

INFO:MainThread:gensim.models.doc2vec:collecting all words and their counts
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #10000, processed 26764263 words (17774127/s), 427 word types, 10000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #20000, processed 53545180 words (18170153/s), 427 word types, 20000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #30000, processed 80339282 words (18032346/s), 427 word types, 30000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #40000, processed 107093186 words (17601275/s), 427 word types, 40000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #50000, processed 133901015 words (18037664/s), 427 word types, 50000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #60000, processed 160664463 words (17839051/s), 427 word types, 60000 tags
INFO:MainThrea

INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #590000, processed 1580682846 words (17951390/s), 427 word types, 590000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #600000, processed 1607533125 words (17767968/s), 427 word types, 600000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #610000, processed 1634342117 words (17764510/s), 427 word types, 610000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #620000, processed 1661071425 words (17976352/s), 427 word types, 620000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #630000, processed 1687914967 words (18005033/s), 427 word types, 630000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #640000, processed 1714676642 words (17787859/s), 427 word types, 640000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #650000, processed 1741425230 words (17843942/s), 427 word types, 650000 tags
INFO:MainThread:gensim.models.doc2vec:PRO

INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #1190000, processed 3187189305 words (17614817/s), 427 word types, 1190000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #1200000, processed 3214026237 words (17371554/s), 427 word types, 1200000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #1210000, processed 3240707637 words (17578046/s), 427 word types, 1210000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #1220000, processed 3267524320 words (17492784/s), 427 word types, 1220000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #1230000, processed 3294306429 words (17577726/s), 427 word types, 1230000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #1240000, processed 3320986570 words (17536733/s), 427 word types, 1240000 tags
INFO:MainThread:gensim.models.doc2vec:PROGRESS: at example #1250000, processed 3347776716 words (17546425/s), 427 word types, 1250000 tags
INFO:MainThread:gensim.mode

In [69]:
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

INFO:MainThread:gensim.models.base_any2vec:training model with 3 workers on 427 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 0.18% examples, 1894130 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 0.40% examples, 1904185 words/s, in_qsize 6, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 0.62% examples, 1903849 words/s, in_qsize 3, out_qsize 2
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 0.81% examples, 1909016 words/s, in_qsize 6, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 1.02% examples, 1910385 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 1.25% examples, 1912084 words/s, in_qsize 4, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 1.44% examples, 1913392 words/s, in_qsi

INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 13.98% examples, 1914775 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 14.17% examples, 1914663 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 14.38% examples, 1914607 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 14.61% examples, 1914767 words/s, in_qsize 6, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 14.80% examples, 1915066 words/s, in_qsize 6, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 15.01% examples, 1915060 words/s, in_qsize 5, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 15.24% examples, 1914954 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 15.42% examples, 1915059 words/s, in_qsize 6, out_qsize 0
INFO:Mai

INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 27.64% examples, 1895487 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 27.84% examples, 1894850 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 28.06% examples, 1894569 words/s, in_qsize 4, out_qsize 2
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 28.24% examples, 1894348 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 28.45% examples, 1893970 words/s, in_qsize 4, out_qsize 2
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 28.67% examples, 1893790 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 28.85% examples, 1893802 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 29.06% examples, 1893784 words/s, in_qsize 6, out_qsize 1
INFO:Mai

INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 41.52% examples, 1897492 words/s, in_qsize 5, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 41.73% examples, 1897519 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 41.95% examples, 1897478 words/s, in_qsize 6, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 42.15% examples, 1897618 words/s, in_qsize 6, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 42.35% examples, 1897621 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 42.57% examples, 1897667 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 42.77% examples, 1897821 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 42.97% examples, 1897782 words/s, in_qsize 4, out_qsize 1
INFO:Mai

INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 55.45% examples, 1899304 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 55.63% examples, 1899336 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 55.84% examples, 1899312 words/s, in_qsize 4, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 56.07% examples, 1899377 words/s, in_qsize 6, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 56.25% examples, 1899368 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 56.46% examples, 1899344 words/s, in_qsize 4, out_qsize 2
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 56.69% examples, 1899411 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 56.87% examples, 1899404 words/s, in_qsize 6, out_qsize 1
INFO:Mai

...

INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 38.74% examples, 1815107 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 38.94% examples, 1815353 words/s, in_qsize 4, out_qsize 2
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 39.12% examples, 1814942 words/s, in_qsize 4, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 39.33% examples, 1814958 words/s, in_qsize 5, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 39.54% examples, 1815061 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 39.72% examples, 1815294 words/s, in_qsize 6, out_qsize 1
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 39.93% examples, 1815465 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 40.15% examples, 1815657 words/s, in_qsize 5, out_qsize 0


INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 51.83% examples, 1816955 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 52.04% examples, 1817065 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 52.26% examples, 1817211 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 52.44% examples, 1817339 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 52.65% examples, 1817455 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 52.87% examples, 1817566 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 53.05% examples, 1817771 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 53.25% examples, 1817882 words/s, in_qsize 6, out_qsize 0


INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 65.06% examples, 1821137 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 65.25% examples, 1821109 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 65.46% examples, 1821157 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 65.66% examples, 1821286 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 65.85% examples, 1821259 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 66.06% examples, 1821332 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 66.27% examples, 1821474 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 66.46% examples, 1821587 words/s, in_qsize 5, out_qsize 0


INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 78.18% examples, 1821510 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 78.39% examples, 1821593 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 78.57% examples, 1821586 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 78.78% examples, 1821566 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 79.00% examples, 1821648 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 79.17% examples, 1821683 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 79.38% examples, 1821787 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 79.60% examples, 1821884 words/s, in_qsize 6, out_qsize 0


INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 91.13% examples, 1818169 words/s, in_qsize 5, out_qsize 2
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 91.30% examples, 1817883 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 91.49% examples, 1817740 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 91.70% examples, 1817607 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 91.87% examples, 1817518 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 92.07% examples, 1817526 words/s, in_qsize 6, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 92.29% examples, 1817533 words/s, in_qsize 5, out_qsize 0
INFO:MainThread:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 92.47% examples, 1817470 words/s, in_qsize 5, out_qsize 0


In [81]:
#INFO:MainThread:gensim.models.base_any2vec:training on a 168268023400 raw words (36737163842 effective words) took 20013.4s, 1835625 effective words/s

In [70]:
x ="Yeah, I'm gonna take my horse to the old town road I'm gonna ride 'til \
I can't no more I'm gonna take my horse to the old town \
road I'm gonna ride 'til I can't no more\
I got the horses in the back\
Horse tack is attached\
Hat is matte black\
Got the boots that's black to match\
Ridin' on a horse, ha\
You can whip your Porsche\
I been in the valley\
You ain't been up off that porch, now"

In [71]:
x = x.split(" ")

In [72]:
y = "Hat down, cross town, livin' like a rockstar\
Spent a lot of money on my brand new guitar\
Baby's got a habit: diamond rings and Fendi sports bras\
Ridin' down Rodeo in my Maserati sports car\
Got no stress, I've been through all that\
I'm like a Marlboro Man so I kick on back\
Wish I could roll on back to that old town road\
I wanna ride 'til I can't no more"

In [73]:
y= y.split(" ")

In [118]:
x = model.infer_vector(x)

In [75]:
model.infer_vector(y)

array([ 0.01619156,  0.06904101, -0.05003458, -0.12973228,  0.19368632,
       -0.11356706,  0.07526851,  0.11501788,  0.07736818, -0.02297388,
       -0.0829546 , -0.2607602 ,  0.17981274, -0.0074904 , -0.24368016,
        0.04624335, -0.02275219,  0.06010182,  0.14539976, -0.07497831,
       -0.057869  ,  0.06439102,  0.05975559, -0.06264416, -0.065087  ,
       -0.03264274, -0.3389467 ,  0.15792006, -0.0051421 ,  0.04789678,
        0.02073748, -0.00315201,  0.05405095,  0.07440678, -0.01841925,
       -0.13446917,  0.0196117 ,  0.05729372, -0.00552276,  0.08972521,
        0.06873091, -0.02580534, -0.06006255, -0.09245363,  0.03733119,
        0.26344448, -0.1375758 , -0.14208677, -0.0529626 , -0.02449816],
      dtype=float32)

In [76]:
pj = "Yamborghini chain, rest in peace to my superior\
Hermès link could feed a village in Liberia\
TMZ taking pictures, causin' mad hysteria\
Momma see me on BET and started tearin' up\
I'ma start killin' niggas, how you get that trife?\
I attended Harlem picnics where you risked your life\
Uncle used to skim work, sellin' nicks at night\
I was only 8 years old, watching Nick at Nite\
Uncle Psycho was in that bathroom buggin'!\
Knife to his guts, hope Daddy don't cut him\
Suicidal thoughts brought to me with no advisory\
He was pitchin' dummy, sellin' fiends mad ivory\
Grandma had the arthritis in her hands, bad!\
She was poppin' pills like rappers in society\
I fuck yo bitch for the irony\
I'll send Meechy at yo ho if yo bitch keep eyein' me"

In [77]:
pj = pj.split(" ")

In [78]:
model.infer_vector(pj)

array([-0.01404641, -0.22817972, -0.05826369,  0.0550184 , -0.05290065,
       -0.29565474,  0.15267037, -0.08073229,  0.11210231,  0.26128456,
       -0.07849698, -0.16467237, -0.12959144, -0.06712817,  0.11950538,
       -0.04022655, -0.10215375, -0.02365132,  0.21611644, -0.20773429,
       -0.38357413,  0.10960454,  0.28275275, -0.03749084, -0.04633784,
        0.16836615,  0.00701733,  0.04535856,  0.10827483, -0.03683288,
        0.09297064, -0.07553069,  0.1868768 ,  0.10294448, -0.04070481,
       -0.11885776, -0.0727586 ,  0.03321617, -0.01311952,  0.00749458,
        0.10874549, -0.10437107, -0.06059659, -0.02722262, -0.11259509,
        0.2535764 , -0.11820628,  0.0368124 , -0.02995643,  0.0605309 ],
      dtype=float32)

In [105]:
pj1 = "I'ma explain why you probably never seen me\
I'm in a sunken place, no Instagram, I'm watchin' TV\
I think I trade my breakfast, lunch and dinner for some kitty\
Please believe me, I see RiRi, I'ma eat it like panini\
I go dumb up in the broad, hit the walls like graffiti\
Indian burns all up on a nigga wee-wee\
I think I need a foursome, Bella, Kendall, Gigi\
It'd be easy if Reneezy hook it all up on the leezy\
I go crazy in my Yeezy, Kirk Kneezy on the beat\
I told 'em now we finna glow up in the street\
Rappers talk subliminal but they don't talk to me\
Put 'em in a Jersey shore like Pauly D"

In [106]:
pj1 = pj1.split(" ")
pj1 = model.infer_vector(pj1)

In [119]:
model.docvecs.most_similar([x],topn=10)

[(1025060, 0.7205559611320496),
 (1210483, 0.6971139311790466),
 (1180525, 0.687134325504303),
 (419918, 0.6739033460617065),
 (1208482, 0.6661245822906494),
 (328680, 0.6597744822502136),
 (921365, 0.6594496965408325),
 (1185653, 0.6560283899307251),
 (974652, 0.6549181938171387),
 (1183485, 0.6514763236045837)]

In [120]:
songs = df.song_identifier.evaluate()

In [121]:
songs[1025060],songs[1210483],songs[1180525],songs[419918],songs[1208482],songs[328680], songs[921365],songs[1185653],songs[1183485]

('Papoose-talkin-sizzurp',
 'Biz-markie-kung-fu',
 'Biz-markie-kung-fu',
 'Big-l-accapella-2-fall-back',
 'Apathy-personal-jesus',
 'Benefit-blind-following',
 'The-roots-get-busy',
 'Rohff-dans-tes-yeux',
 'Mc-solaar-bouge-de-la')

In [122]:
model.save("lyric_doc2vec")

INFO:MainThread:gensim.utils:saving Doc2Vec object under lyric_doc2vec, separately None
INFO:MainThread:gensim.utils:storing np array 'vectors_docs' to lyric_doc2vec.docvecs.vectors_docs.npy
INFO:MainThread:gensim.utils:storing np array 'vectors_docs_norm' to lyric_doc2vec.docvecs.vectors_docs_norm.npy
/home/cwcollins/.local/share/virtualenvs/music_recommender-A4vLtpgt/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
INFO:MainThread:gensim.utils:saved lyric_doc2vec


In [125]:
model2=Doc2Vec.load("lyric_doc2vec")

INFO:MainThread:gensim.utils:loading Doc2Vec object from lyric_doc2vec
/home/cwcollins/.local/share/virtualenvs/music_recommender-A4vLtpgt/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
INFO:MainThread:gensim.utils:loading vocabulary recursively from lyric_doc2vec.vocabulary.* with mmap=None
INFO:MainThread:gensim.utils:loading trainables recursively from lyric_doc2vec.trainables.* with mmap=None
INFO:MainThread:gensim.utils:loading wv recursively from lyric_doc2vec.wv.* with mmap=None
INFO:MainThread:gensim.utils:loading docvecs recursively from lyric_doc2vec.docvecs.* with mmap=None
INFO:MainThread:gensim.utils:loading vectors_docs from lyric_doc2vec.docvecs.vectors_docs.npy with mmap=None
INF

In [126]:
model2.docvecs.most_similar([x],topn=10)

[(1025060, 0.7205559611320496),
 (1210483, 0.6971139311790466),
 (1180525, 0.687134325504303),
 (419918, 0.6739033460617065),
 (1208482, 0.6661245822906494),
 (328680, 0.6597744822502136),
 (921365, 0.6594496965408325),
 (1185653, 0.6560283899307251),
 (974652, 0.6549181938171387),
 (1183485, 0.6514763236045837)]